In [ ]:
using DrWatson
@quickactivate "masterarbeit"

In [ ]:
using Random
using LaTeXStrings
using GLMakie
using StatsBase

In [ ]:
Makie.inline!(true)
fontsize_theme = Theme(fontsize=35)
set_theme!(fontsize_theme)
wblue = Makie.wong_colors()[1]
worange = Makie.wong_colors()[2]
wgreen = Makie.wong_colors()[3]
wpink = Makie.wong_colors()[4]
wlblue = Makie.wong_colors()[5]
worange = Makie.wong_colors()[6]
wyellow = Makie.wong_colors()[7];

In [ ]:
using Revise

In [ ]:
using masterarbeit

In [ ]:
include(joinpath(projectdir(),"src/vegas.jl"))

In [ ]:
f = v -> double_gauss(v)[1]
vegasstart = [0.0, 0.0]
vegasend = [1.0, 1.0]
xticks = [0.0, 0.25, 0.5, 0.75, 1.0]
yticks = [0.0, 0.25, 0.5, 0.75, 1.0]
xs = LinRange(0.0, 1.0, 1001)
ys = LinRange(0.0, 1.0, 1001)
fevals = sample -> f([sample[1], sample[2]]);

In [ ]:
res = vegas(f, vegasstart, vegasend)
weights = res.weights;

In [ ]:
fig = Figure(size=(1000,1000), figure_padding=40)
ax2 = Axis(fig[1,1], xticks=xticks, yticks=yticks, 
    aspect=1, xlabel=L"x_1", ylabel=L"x_2", xlabelsize=50, ylabelsize=50)
zs = [f([x,y])[1] for x in xs, y in ys]
zs_normalized = zs ./ 0.232324
hm = heatmap!(xs, ys, zs_normalized, colorrange=(0,5))
vlines!(res.adaptive_grid[:,1][1:50:end], color=:white, linewidth=2)
hlines!(res.adaptive_grid[:,2][1:50:end], color=:white, linewidth=2)
#fig[1, 2] = GridLayout(width = 20)
#Colorbar(fig[1,3], hm, width=40)
save("doublegauss2d_vegas_grid.png", fig)
fig

In [ ]:
#=fig = Figure(size=(1000,1000), figure_padding=40)
ax2 = Axis(fig[1,1], xticks=xticks, yticks=yticks, 
    aspect=1, xlabel=L"x_1", ylabel=L"x_2")
vlines!(res.adaptive_grid[:,1][1:50:end], color=:black, linewidth=1)
hlines!(res.adaptive_grid[:,2][1:50:end], color=:black, linewidth=1)
save("doublegauss2d_vegas_grid.png", fig)
fig=#

In [ ]:
samples, Js = sample_from_adaptive_grid(res, 10^7)
weights = Js .* fevals.(samples);

In [ ]:
sx = [s[1] for s in samples]
sy = [s[2] for s in samples];

In [ ]:
histo = fit(Histogram, (sx, sy), nbins=100)
histo_n = StatsBase.normalize(histo, mode=:pdf)
fig = Figure(size=(1200,1000), figure_padding=40)
ax = Axis(fig[1,1], xlabel=L"x_1", ylabel=L"x_2", xticks=xticks, yticks=yticks, aspect=1, xlabelsize=50, ylabelsize=50)
hm = heatmap!(histo.edges[1], histo.edges[2], histo_n.weights, colorrange=(0,5))
fig[1, 2] = GridLayout(width = 20)
Colorbar(fig[1,3], hm, width=40)
save("doublegauss2d_vegas_samples.png", fig)
fig

In [ ]:
fig = Figure(size=(1500,1000))
ax = Axis(fig[1,1], xlabel=L"x_1", xticks=xticks, aspect=1, xlabelsize=50)
lines!(xs, [f([x])[1] for x in xs] ./ 0.68165, color=worange, label="ground truth")
stephist!([s[1] for s in samples], bins=40, normalization=:pdf, color=wblue, linewidth=4, label="sample distribution")
#fig[1,2] = Legend(fig, ax)
save("doublegauss2d_vegas_compare.png", fig)
fig

In [ ]:
mcint = res.integral_estimate
mcerror = res.standard_deviation
println("mc integral = $mcint")
println("standard deviation = $mcerror")

In [ ]:
w_n = weights ./ mcint
w_avg = mean(w_n)
w_max = maximum(w_n)
efficiency = w_avg / w_max
println("mean weight = $w_avg")
println("max weight = $w_max")
println("unweighting efficiency = $efficiency")

In [ ]:
#=fig = Figure(size=(1000,700))
ax = Axis(fig[1,1], ylabel=L"N", xlabel=L"w_{i,\mathrm{rel}}", yscale=Makie.pseudolog10, yticks=[0, 10^1, 10^3, 10^5, 10^7])
stephist!(wi, bins=100)
save("doublegauss2d_vegas_weights.png", fig)
fig=#

In [ ]:
using JLD2

In [ ]:
save_object("vegas_weights.jld2", w_n)